## Chapter 5: Dice pools

Where a die is a probability distribution over outcomes, a dice `Pool` is a probability distribution over the outcomes of several dice---
in other words, a probability distribution over [multisets](https://en.wikipedia.org/wiki/Multiset).

### Pool creation

To create a pool where all of the dice are the same, you can use the `pool` method on a die.

In [1]:
import piplite
await piplite.install("icepool")

from icepool import d6

# 4d6
print(d6.pool(4))

Pool of 4 dice with keep_tuple=(1, 1, 1, 1)
  Die({1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})
  Die({1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})
  Die({1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})
  Die({1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})



You can also construct a pool of different types of dice using the `Pool` constructor. For example, here is 2d4 and 2d6 in a pool.

In [2]:
from icepool import d4, Pool

print(Pool([d4, d4, d6, d6]))

Pool of 4 dice with keep_tuple=(1, 1, 1, 1)
  Die({1: 1, 2: 1, 3: 1, 4: 1})
  Die({1: 1, 2: 1, 3: 1, 4: 1})
  Die({1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})
  Die({1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})



Note that a pool is only a description of how to roll dice. To actually perform an evaluation, you need to either use one of the built-in pool methods, or a custom `OutcomeCountEvaluator`.

The most common evaluation is summing, which you can do using the `Pool.sum()` method.

In [3]:
print(Pool([d4, d4, d6, d6]).sum())

Die with denominator 576

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       4 |        1 |   0.173611% |
|       5 |        4 |   0.694444% |
|       6 |       10 |   1.736111% |
|       7 |       20 |   3.472222% |
|       8 |       33 |   5.729167% |
|       9 |       48 |   8.333333% |
|      10 |       62 |  10.763889% |
|      11 |       72 |  12.500000% |
|      12 |       76 |  13.194444% |
|      13 |       72 |  12.500000% |
|      14 |       62 |  10.763889% |
|      15 |       48 |   8.333333% |
|      16 |       33 |   5.729167% |
|      17 |       20 |   3.472222% |
|      18 |       10 |   1.736111% |
|      19 |        4 |   0.694444% |
|      20 |        1 |   0.173611% |




### Keeping dice

Outcomes of individual dice come out of a pool in sorted order. You can pick a die from a particular sorted index, drop dice from sorted positions, count dice multiple times, or even count dice negative times using the subscript operator. Here are some examples:

In [4]:
# 4d6 drop lowest.
print(d6.pool(4)[-3:].sum())

# Other equivalents.

# Give one integer per sorted position.
d6.pool(4)[0, 1, 1, 1].sum()

# Ellipses will fill with zero counts.
d6.pool(4)[..., 1, 1, 1].sum()

# A single index produces a Die directly.

# What was the value of the dropped die?
print(d6.pool(4)[0])

Die with denominator 1296

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       3 |        1 |   0.077160% |
|       4 |        4 |   0.308642% |
|       5 |       10 |   0.771605% |
|       6 |       21 |   1.620370% |
|       7 |       38 |   2.932099% |
|       8 |       62 |   4.783951% |
|       9 |       91 |   7.021605% |
|      10 |      122 |   9.413580% |
|      11 |      148 |  11.419753% |
|      12 |      167 |  12.885802% |
|      13 |      172 |  13.271605% |
|      14 |      160 |  12.345679% |
|      15 |      131 |  10.108025% |
|      16 |       94 |   7.253086% |
|      17 |       54 |   4.166667% |
|      18 |       21 |   1.620370% |


Die with denominator 1296

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |      671 |  51.774691% |
|       2 |      369 |  28.472222% |
|       3 |      175 |  13.503086% |
|       4 |       65 |   5.015432% |
|       5 |       15 |   1.157407% |
|       6 |        

### Expansion

If you want to know all of the possible sorted rolls from a pool, you can use the `expand` method, which produces a `Die` whose outcomes are tuples containing the sorted rolls. For example, here is the highest two out of three d6s:

In [5]:
print(d6.pool(3)[0, 1, 1].expand())

Die with denominator 216

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          1 |          1 |        1 |   0.462963% |
|          1 |          2 |        3 |   1.388889% |
|          1 |          3 |        3 |   1.388889% |
|          1 |          4 |        3 |   1.388889% |
|          1 |          5 |        3 |   1.388889% |
|          1 |          6 |        3 |   1.388889% |
|          2 |          2 |        4 |   1.851852% |
|          2 |          3 |        9 |   4.166667% |
|          2 |          4 |        9 |   4.166667% |
|          2 |          5 |        9 |   4.166667% |
|          2 |          6 |        9 |   4.166667% |
|          3 |          3 |        7 |   3.240741% |
|          3 |          4 |       15 |   6.944444% |
|          3 |          5 |       15 |   6.944444% |
|          3 |          6 |       15 |   6.944444% |
|          4 |          4 |       10 |   4.629630% |
|          4 |      

`expand` is not particularly efficient if you are keeping a lot of dice. Unless you are really ultimately interested in the entire sequence, use a built-in pool method or `OutcomeCountEvaluator` instead.

### More multiset operations

There's also a variety of methods for modifying the count of each outcome generated by a pool before doing an evaluation. Here are some examples:

The `unique()` method keeps each outcome at most once.

In [6]:
print(d6.pool(3).unique().count())

Die with denominator 216

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |        6 |   2.777778% |
|       2 |       90 |  41.666667% |
|       3 |      120 |  55.555556% |




The `>=` operator checks whether the roll of the left pool is a superset of the right. For example, this checks whether the roll of a pool has at least one 1 and one 2.

In [7]:
print(d6.pool(3) >= [1, 2])

Die with denominator 216

| Outcome | Quantity | Probability |
|:--------|---------:|------------:|
| False   |      186 |  86.111111% |
| True    |       30 |  13.888889% |




Note that sequences and mappings are automatically converted into pools that always roll that multiset of elements.

The `&` operator takes the intersection of two pools---that is, all matching pairs of one outcome from the left side and one from the right side. For example, here's the number of matching pairs that can be formed from two pools of 3d6.

In [8]:
print((d6.pool(3) & d6.pool(3)).count())

Die with denominator 46656

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |     9750 |  20.897634% |
|       1 |    23580 |  50.540123% |
|       2 |    12330 |  26.427469% |
|       3 |      996 |   2.134774% |




You can find more [Pool methods](https://highdiceroller.github.io/icepool/apidoc/icepool.html#Pool) in the API documentation, including those inherited from superclasses. These include extracting unique outcomes, finding matching sets and straights, and more.